# Generating signals

Example on how to generate continuous gravitational-wave signals.

0. [Introduction](#introduction)
0. [Generating an ensemble of signals](#ensemble)

In [ ]:
import numpy as np

import pyfstat
from pyfstat.utils import get_sft_as_arrays

# Local module to simplify plotting
import tutorial_utils

logger = pyfstat.set_up_logger(label="1_generating_signals", log_level="INFO")

%matplotlib inline

## Introduction<a name="the-sft-format"></a>
Continuous gravitational-wave signals (CWs) are long-lasting forms of gravitational radiation.
They are usually characterized using two sets of parameters, namely the *amplitude* parameters
and the *Doppler* parameters, respectively referred to as $\mathcal{A}$ and $\lambda$.

For the typical case of a rapidly-spinning neutron star, the amplitude parameter contain the nominal 
CW amplitude $h_0$, the (cosine of the) source inclination angle with respect to the line of sight $\cos\iota$,
the polarization angle $\psi$ and the initial phase of the wave $\psi$.
Depending on the emission mechanism, $h_0$ can be further described using further physical quantities such as
the source's frequency, ellipticity, or distance to the detector.

Doppler parameters describe the evolution of the gravitational-wave frequency both due to physical processes
undergoing at the source and the motion of the interferometric detector with respect to the 
Solar system barycenter (i.e. the Sun). In this tutorial, we will limit ourselves to gravitational wave
frequency $f_0$, spindown $f_1$ and sky position $\hat{n}$, which we will parametrize using the right ascension
$\alpha$ and declination $\delta$ angles.

A detailed explanation of these parameters can be found in
[this technical document](https://dcc.ligo.org/LIGO-T0900149/public).

`pyfstat.Writer` allows these variables to be given as inputs to produce SFTs containing a CW signal and
(optionally) Gaussian noise. Signal parameters are both available as attributes in the `pyfstat.Writer` instance
as as a `.cff` file in `outdir`. 
Alternatively, as exemplified in [PyFstat_example_injecting_into_noise_sfts.py](../other_examples),
the `noiseSFTs` can be used to provide a pre-generated set of SFTs as background noise.

In [ ]:
writer_kwargs = {
    "label": "single_detector_gaussian_noise",
    "outdir": "PyFstat_example_data",
    "tstart": 1238166018,
    "duration": 365 * 86400,
    "detectors": "H1",
    "sqrtSX": 1e-23,
    "Tsft": 1800,
    "SFTWindowType": "tukey",
    "SFTWindowBeta": 0.01,
}

signal_parameters = {
    "F0": 100.0,
    "F1": -1e-9,
    "Alpha": 0.0,
    "Delta": 0.0,
    "h0": 1e-22,
    "cosi": 1,
    "psi": 0.0,
    "phi": 0.0,
    "tref": writer_kwargs["tstart"],
}

writer = pyfstat.Writer(**writer_kwargs, **signal_parameters)
writer.make_data()
frequency, timestamps, fourier_data = get_sft_as_arrays(writer.sftfilepath)

In [ ]:
tutorial_utils.plot_real_imag_spectrograms(
    timestamps["H1"], frequency, fourier_data["H1"]
)
fig, ax = tutorial_utils.plot_amplitude_phase_spectrograms(
    timestamps["H1"], frequency, fourier_data["H1"]
);

These plots show the typical features of CW signals.

Short-scale amplitude modulations are due to the rotation of the Earth, 
as the detector has different sensitivities depending on its orientation.

The frequency is modulated in a quasi-periodic fashion due to the translation of the interferometric
detector around the Sun. The downwards trend is due to the presence of a negative source spindown due to the
emission of energy as gravitational waves.

The instantaneous frequency of a CW at the detector can be expressed as
$$
f(t) = \left[f_0 + f_1 \left(t - t_{\text{ref}}\right)\right]\left[1 + \frac{\vec{v} \cdot \hat{n}}{c}\right],
$$
where $\vec{v}/c$ is the velocity of the detector normalized by the speed of light in vaccum 
and $t_{\text{ref}}$ is a fiducial reference time at which $f_0$ and $f_1$ are specified.

Detector velocities can be retrieved from SFT files using `pyfstat.DetectorStates`:

In [ ]:
states = pyfstat.DetectorStates().get_multi_detector_states_from_sfts(
    writer.sftfilepath, central_frequency=writer.F0, time_offset=0
)

ts = np.array([data.tGPS.gpsSeconds for data in states.data[0].data])
velocities = np.vstack([data.vDetector for data in states.data[0].data]).T

n = np.array(
    [
        [
            np.cos(writer.Alpha) * np.cos(writer.Delta),  # Cartesian X
            np.sin(writer.Alpha) * np.cos(writer.Delta),  # Cartesian Y
            np.sin(writer.Delta),  # Cartesian Z
        ]
    ]
)

f_inst = ((writer.F0 + (ts - writer.tref) * writer.F1) * (1 + np.dot(n, velocities)))[0]

for a in ax:
    for off in [-2, 2]:
        a.plot(
            (ts - ts[0]) / 1800,
            f_inst + off / writer.Tsft,
            color="white",
            ls="--",
            label="Instantaneous frequency" if off == 2 else "",
        )
    a.legend()
fig

The presence of visible CW features in the spectrogram can be quantified using the so-called
[sensitivity depth](https://arxiv.org/abs/1808.02459), which is defined as the ratio between the noise
and CW amplitude
$$
\mathcal{D} = \frac{\sqrt{S_{\text{n}}}}{h_0}\;.
$$

Current searches for CW signals from unknown sources are able to detect a sensitivity depth ranging between
$10\;\text{Hz}^{-1/2}$ and $50\;\text{Hz}^{-1/2}$. Visual features, however, tend to disappear around a depth
of $20\;\text{Hz}^{-1/2}$.

In [ ]:
# CW signal at a depth of 20
signal_parameters["h0"] = writer_kwargs["sqrtSX"] / 20.0

writer = pyfstat.Writer(**writer_kwargs, **signal_parameters)

# Create SFTs
writer.make_data()

frequency, timestamps, fourier_data = get_sft_as_arrays(writer.sftfilepath)

In [ ]:
tutorial_utils.plot_real_imag_spectrograms(
    timestamps["H1"], frequency, fourier_data["H1"]
)
tutorial_utils.plot_amplitude_phase_spectrograms(
    timestamps["H1"], frequency, fourier_data["H1"]
);

Alternatively, CW signals can be characterized in terms of their (squared) signal-to-noise ratio (SNR) $\rho^2$.
As opposed to sensitivity depth, which focuses on instantaneous features of the CW, $\rho^2$ is an integrated
quantity along the full duration of the observing run. 
The dependency of the *optimal* SNR (i.e. assuming the Doppler parameters match those of the signal) depends in
a non-trivial way on the amplitude parameters and sky position of the source (due to the anisotropy of the
detector response function) [[Eq. (77)](https://dcc.ligo.org/LIGO-T0900149/public)]; 
the average optimal SNR for a uniform distribution of sources across the sky with an isotropic polarization angle, however, 
can be readily expressed as
$$
\langle \rho^2 \rangle_{\vec{n}, \psi} = 
\frac{1}{20}
\frac{h_0^2 \left(\cos^4\iota + 6 \cos^2\iota + 1\right)}
{S_{\textrm{n}}}
T_{\textrm{obs}},
$$
where $T_{\textrm{obs}}$ represents full duration of the data stream.

The optimal SNR for a specific template can be computed using `pyfstat.SignalToNoiseRatio` assuming Gaussian noise with a given
`sqrtSX` value or from a specific set of SFTs.

In [ ]:
snr = pyfstat.SignalToNoiseRatio.from_sfts(F0=writer.F0, sftfilepath=writer.sftfilepath)
squared_snr = snr.compute_snr2(
    Alpha=writer.Alpha,
    Delta=writer.Delta,
    psi=writer.psi,
    phi=writer.phi,
    h0=writer.h0,
    cosi=writer.cosi,
)
print(f"SNR: {np.sqrt(squared_snr)}")

## Generating an ensemble of signals<a name="ensemble"></a>

Characterizing a CW search or follow-up method usually involves analyzing its response to an ensemble of signals
from a population of interest.
For the case of all-sky searches, this population is usually consistent with a uniform distribution of sources
across the sky with isotropic orientation.

The `pyfstat.InjectionParametersGenerator` provides methods to draw parameters 
from generic distributions in a suitable format, ready to be fed into `pyfstat.Writer`.
Uniform sampling across the sky is baked in the child class `pyfstat.AllSkyInjectionParametersGenerator`,
and isotropic priors on amplitude parameters are available in 
`pyfstat.isotropic_amplitude_distribution`.

All-sky searches tend to perform injection campaigns at fixed values of `h0` (or, equivalently, $\mathcal{D}$).

In [ ]:
# Generate 10 signals with parameters drawn from a specific population
num_signals = 10

writer_kwargs = {
    "tstart": 1238166018,
    "duration": 365 * 86400,
    "detectors": "H1",
    "sqrtSX": 1e-23,
    "Tsft": 1800,
    "SFTWindowType": "tukey",
    "SFTWindowBeta": 0.01,
}

signal_parameters_generator = pyfstat.AllSkyInjectionParametersGenerator(
    priors={
        "F0": {"stats.uniform": {"loc": 100.0, "scale": 0.1}},
        "F1": -1e-10,
        "F2": 0,
        "h0": writer_kwargs["sqrtSX"] / 10,  # Fix amplitude at depth 10.
        **pyfstat.isotropic_amplitude_distribution,
        "tref": writer_kwargs["tstart"],
    },
)

for ind in range(num_signals):

    params = signal_parameters_generator.draw()
    writer_kwargs["outdir"] = f"PyFstat_example_data_ensemble/Signal_{ind}"
    writer_kwargs["label"] = f"Signal_{ind}"

    writer = pyfstat.Writer(**writer_kwargs, **params)
    writer.make_data()